In [1]:
import heapq

class PuzzleState:
    def __init__(self, state, parent=None, move=None, depth=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = 0
        self.calculate_cost()

    def __lt__(self, other):
        return self.cost < other.cost

    def calculate_cost(self):
        self.cost = self.depth + self.calculate_heuristic()

    def calculate_heuristic(self):
        # Manhatten distance heuristic
        total = 0
        for i in range(3):
            for j in range(3):
                if self.state[i][j] != 0:
                    row, col = divmod(self.state[i][j] - 1, 3)
                    total += abs(row - i) + abs(col - j)
        return total

    def __eq__(self, other):
        return self.state == other.state

    def __hash__(self):
        return hash(str(self.state))

    def __repr__(self):
        return f'{self.state}'

    def generate_next_states(self):
        zero_row, zero_col = self.find_zero_position()
        next_states = []

        for move in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            new_row, new_col = zero_row + move[0], zero_col + move[1]
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_state = [row[:] for row in self.state]
                new_state[zero_row][zero_col] = new_state[new_row][new_col]
                new_state[new_row][new_col] = 0
                next_states.append(PuzzleState(new_state, self, move, self.depth + 1))

        return next_states

    def find_zero_position(self):
        for i in range(3):
            for j in range(3):
                if self.state[i][j] == 0:
                    return i, j


def a_star_search(initial_state, goal_state):
    frontier = []
    explored = set()

    heapq.heappush(frontier, initial_state)

    while frontier:
        current_state = heapq.heappop(frontier)

        if current_state.state == goal_state.state:
            return current_state

        explored.add(current_state)

        for next_state in current_state.generate_next_states():
            if next_state not in frontier and next_state not in explored:
                heapq.heappush(frontier, next_state)
            elif next_state in frontier:
                # Find index of the existing state in the frontier
                existing_index = frontier.index(next_state)
                existing_state = frontier[existing_index]

                # Update the frontier if the new state has a lower cost
                if next_state.cost < existing_state.cost:
                    frontier[existing_index] = next_state
                    heapq.heapify(frontier)

    return None

def print_solution(solution):
    if solution:
        moves = []
        current_state = solution
        while current_state.parent:
            moves.insert(0, current_state.move)
            current_state = current_state.parent
        print("Solution found in", len(moves), "moves:")
        for move in moves:
            if move == (0, 1):
                print("Move right")
            elif move == (0, -1):
                print("Move left")
            elif move == (1, 0):
                print("Move down")
            elif move == (-1, 0):
                print("Move up")
        print("Final state:")
        print(solution.state)
    else:
        print("No solution found.")

if __name__ == "__main__":
    initial_state = [[1, 2, 3],
                     [4, 0, 6],
                     [7, 5, 8]]

    goal_state = [[0, 4, 3],
                  [2, 1, 6],
                  [7, 5, 8]]

    initial_node = PuzzleState(initial_state)
    goal_node = PuzzleState(goal_state)

    solution = a_star_search(initial_node, goal_node)
    print_solution(solution)

Solution found in 6 moves:
Move up
Move left
Move down
Move right
Move up
Move left
Final state:
[[0, 4, 3], [2, 1, 6], [7, 5, 8]]
